In [1]:
import tensorflow as tf
import numpy as np
from time import time
import os

print(tf.__version__)

2.8.0


### Checkpoint function
> - checkpoint는 말 그대로 checkpoint
> - 학습이 도중에 중단되었을 때, 이어서 할 지점이 되어줌
> - 또는 학습 중간의 모델 성능을 확인할 때도 사용됨

In [2]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")
    
    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt:
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

### Data load & pre-processing function

In [3]:
def load_mnist():
    (train_data, train_labels), (test_data, test_labels) = tf.keras.datasets.mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1)  # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1)  # [N, 28, 28] -> [N, 28, 28, 1]
    
    train_data, test_data = normalize(train_data, test_data)
    
    train_labels = tf.keras.utils.to_categorical(train_labels, 10)  # [N,] -> [N, 10]
    test_labels = tf.keras.utils.to_categorical(test_labels, 10)  # [N,] -> [N, 10]
    
    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0
    
    return train_data, test_data

### Performance function

In [4]:
def loss_function(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits,
                                                                  y_true=labels,
                                                                  from_logits=True))
    return loss

def accuracy_function(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_function(model, images, labels)
    return tape.gradient(loss, model.variables)

### Model function

In [5]:
def faltten():
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init):
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def sigmoid():
    return tf.keras.layers.Activation(tf.keras.activations.sigmoid)

### Create model (class version)

In [18]:
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.RandomNormal()
        
        self.model = tf.keras.Sequential()
        self.model.add(tf.keras.layers.Flatten())
        
        for i in range(2):
            self.model.add(dense(256, weight_init))
            self.model.add(sigmoid())
        
        self.model.add(dense(label_dim, weight_init))
    
    def call(self, x, training=None, mask=None):
        x = self.model(x)
        
        return x

### Define data & hyper-parameter

In [19]:
""" dataset """
train_x,  train_y, test_x, test_y = load_mnist()

""" hyper parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
                                shuffle(buffer_size=100000).\
                                prefetch(buffer_size=batch_size).\
                                batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
                                shuffle(buffer_size=100000).\
                                prefetch(buffer_size=len(test_x)).\
                                batch(len(test_x))

### Define model & optimizer & writer

In [21]:
""" Model """
network = create_model_class(label_dim)

""" Training """
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

""" Writer """
checkpoint_dir = "checkpoints"
logs_dir = "logs"

model_dir = "nn_softmax"

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

### Restore checkpoint & start train or test phase

In [23]:
if train_flag:
    checkpoint = tf.train.Checkpoint(dnn=network)
    
    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()
    
    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)
    
    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)
        counter = checkpoint_counter
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables))
                
                train_loss = loss_function(network, train_input, train_label)
                train_accuracy = accuracy_function(network, train_input, train_label)
                
                for test_input, test_label in test_dataset:
                    test_accuracy = accuracy_function(network, test_input, test_label)
                
                tf.summary.scalar(name="train_loss", data=train_loss, step=counter)
                tf.summary.scalar(name="train_accuracy", data=train_accuracy, step=counter)
                tf.summary.scalar(name="test_accuracy", data=test_accuracy, step=counter)
                
                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy)
                )
                counter += 1
        checkpoint.save(file_prefix=checkpoint_prefix + "-{}".format(counter))

# test phase
else:
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:
        test_accuracy = accuracy_function(network, test_input, test_label)
    
    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...
Epoch: [ 0] [    0/  468] time: 0.3475, train_loss: 2.26735592, train_accuracy: 0.1641, test_Accuracy: 0.0951
Epoch: [ 0] [    1/  468] time: 0.4640, train_loss: 2.30478859, train_accuracy: 0.1172, test_Accuracy: 0.0974
Epoch: [ 0] [    2/  468] time: 0.5801, train_loss: 2.29011011, train_accuracy: 0.1250, test_Accuracy: 0.1538
Epoch: [ 0] [    3/  468] time: 0.7445, train_loss: 2.24152565, train_accuracy: 0.1406, test_Accuracy: 0.1032
Epoch: [ 0] [    4/  468] time: 0.8664, train_loss: 2.27833509, train_accuracy: 0.2031, test_Accuracy: 0.2241
Epoch: [ 0] [    5/  468] time: 0.9868, train_loss: 2.26850533, train_accuracy: 0.1875, test_Accuracy: 0.2182
Epoch: [ 0] [    6/  468] time: 1.1165, train_loss: 2.23803139, train_accuracy: 0.2422, test_Accuracy: 0.2162
Epoch: [ 0] [    7/  468] time: 1.3054, train_loss: 2.23035693, train_accuracy: 0.2578, test_Accuracy: 0.2115
Epoch: [ 0] [    8/  468] time: 1.4391,

Epoch: [ 0] [   75/  468] time: 9.8971, train_loss: 0.99402285, train_accuracy: 0.7109, test_Accuracy: 0.7774
Epoch: [ 0] [   76/  468] time: 10.0508, train_loss: 0.96206683, train_accuracy: 0.7266, test_Accuracy: 0.7841
Epoch: [ 0] [   77/  468] time: 10.1526, train_loss: 0.89982951, train_accuracy: 0.7578, test_Accuracy: 0.7886
Epoch: [ 0] [   78/  468] time: 10.2529, train_loss: 0.84316993, train_accuracy: 0.7812, test_Accuracy: 0.7976
Epoch: [ 0] [   79/  468] time: 10.3518, train_loss: 0.77964699, train_accuracy: 0.8281, test_Accuracy: 0.8078
Epoch: [ 0] [   80/  468] time: 10.5272, train_loss: 0.85175997, train_accuracy: 0.7656, test_Accuracy: 0.8153
Epoch: [ 0] [   81/  468] time: 10.6267, train_loss: 0.86568618, train_accuracy: 0.8047, test_Accuracy: 0.8194
Epoch: [ 0] [   82/  468] time: 10.7251, train_loss: 0.87254107, train_accuracy: 0.7578, test_Accuracy: 0.8175
Epoch: [ 0] [   83/  468] time: 10.8275, train_loss: 0.85512352, train_accuracy: 0.7734, test_Accuracy: 0.8151
Ep

Epoch: [ 0] [  150/  468] time: 18.1441, train_loss: 0.49133614, train_accuracy: 0.8516, test_Accuracy: 0.8707
Epoch: [ 0] [  151/  468] time: 18.2288, train_loss: 0.39432880, train_accuracy: 0.9141, test_Accuracy: 0.8748
Epoch: [ 0] [  152/  468] time: 18.3130, train_loss: 0.46566477, train_accuracy: 0.8906, test_Accuracy: 0.8791
Epoch: [ 0] [  153/  468] time: 18.4861, train_loss: 0.48017514, train_accuracy: 0.8906, test_Accuracy: 0.8820
Epoch: [ 0] [  154/  468] time: 18.5743, train_loss: 0.49403688, train_accuracy: 0.8438, test_Accuracy: 0.8825
Epoch: [ 0] [  155/  468] time: 18.6610, train_loss: 0.43112481, train_accuracy: 0.9219, test_Accuracy: 0.8831
Epoch: [ 0] [  156/  468] time: 18.7513, train_loss: 0.54730260, train_accuracy: 0.8828, test_Accuracy: 0.8843
Epoch: [ 0] [  157/  468] time: 18.9070, train_loss: 0.58512402, train_accuracy: 0.8281, test_Accuracy: 0.8840
Epoch: [ 0] [  158/  468] time: 18.9944, train_loss: 0.49826130, train_accuracy: 0.8750, test_Accuracy: 0.8842
E

Epoch: [ 0] [  224/  468] time: 25.9069, train_loss: 0.32471734, train_accuracy: 0.8906, test_Accuracy: 0.9030
Epoch: [ 0] [  225/  468] time: 25.9966, train_loss: 0.37081409, train_accuracy: 0.8828, test_Accuracy: 0.9028
Epoch: [ 0] [  226/  468] time: 26.0824, train_loss: 0.43541297, train_accuracy: 0.8906, test_Accuracy: 0.9042
Epoch: [ 0] [  227/  468] time: 26.2383, train_loss: 0.41163182, train_accuracy: 0.8672, test_Accuracy: 0.9039
Epoch: [ 0] [  228/  468] time: 26.3239, train_loss: 0.32346499, train_accuracy: 0.9141, test_Accuracy: 0.9031
Epoch: [ 0] [  229/  468] time: 26.4087, train_loss: 0.32933363, train_accuracy: 0.8984, test_Accuracy: 0.9014
Epoch: [ 0] [  230/  468] time: 26.5339, train_loss: 0.22047435, train_accuracy: 0.9688, test_Accuracy: 0.9003
Epoch: [ 0] [  231/  468] time: 26.6153, train_loss: 0.42068022, train_accuracy: 0.8672, test_Accuracy: 0.9005
Epoch: [ 0] [  232/  468] time: 26.7011, train_loss: 0.36432800, train_accuracy: 0.8906, test_Accuracy: 0.9011
E

Epoch: [ 0] [  299/  468] time: 33.6631, train_loss: 0.31178302, train_accuracy: 0.9141, test_Accuracy: 0.9084
Epoch: [ 0] [  300/  468] time: 33.7662, train_loss: 0.34222978, train_accuracy: 0.9297, test_Accuracy: 0.9090
Epoch: [ 0] [  301/  468] time: 33.8527, train_loss: 0.20851600, train_accuracy: 0.9531, test_Accuracy: 0.9089
Epoch: [ 0] [  302/  468] time: 33.9405, train_loss: 0.35225743, train_accuracy: 0.8984, test_Accuracy: 0.9091
Epoch: [ 0] [  303/  468] time: 34.0253, train_loss: 0.29983303, train_accuracy: 0.9219, test_Accuracy: 0.9093
Epoch: [ 0] [  304/  468] time: 34.1657, train_loss: 0.36568299, train_accuracy: 0.8984, test_Accuracy: 0.9101
Epoch: [ 0] [  305/  468] time: 34.2507, train_loss: 0.29547837, train_accuracy: 0.9141, test_Accuracy: 0.9115
Epoch: [ 0] [  306/  468] time: 34.3365, train_loss: 0.32624385, train_accuracy: 0.8906, test_Accuracy: 0.9121
Epoch: [ 0] [  307/  468] time: 34.5292, train_loss: 0.45834500, train_accuracy: 0.9141, test_Accuracy: 0.9133
E

Epoch: [ 0] [  373/  468] time: 41.0794, train_loss: 0.18656924, train_accuracy: 0.9453, test_Accuracy: 0.9192
Epoch: [ 0] [  374/  468] time: 41.1707, train_loss: 0.28886399, train_accuracy: 0.9375, test_Accuracy: 0.9188
Epoch: [ 0] [  375/  468] time: 41.2510, train_loss: 0.35756969, train_accuracy: 0.8906, test_Accuracy: 0.9193
Epoch: [ 0] [  376/  468] time: 41.3373, train_loss: 0.32288361, train_accuracy: 0.8828, test_Accuracy: 0.9190
Epoch: [ 0] [  377/  468] time: 41.4791, train_loss: 0.33011490, train_accuracy: 0.9219, test_Accuracy: 0.9187
Epoch: [ 0] [  378/  468] time: 41.5657, train_loss: 0.25669998, train_accuracy: 0.9453, test_Accuracy: 0.9188
Epoch: [ 0] [  379/  468] time: 41.6484, train_loss: 0.27394760, train_accuracy: 0.9062, test_Accuracy: 0.9192
Epoch: [ 0] [  380/  468] time: 41.7330, train_loss: 0.24912634, train_accuracy: 0.9375, test_Accuracy: 0.9190
Epoch: [ 0] [  381/  468] time: 41.8663, train_loss: 0.39763182, train_accuracy: 0.8984, test_Accuracy: 0.9190
E

Epoch: [ 0] [  447/  468] time: 48.2428, train_loss: 0.27490520, train_accuracy: 0.8906, test_Accuracy: 0.9196
Epoch: [ 0] [  448/  468] time: 48.3291, train_loss: 0.24180919, train_accuracy: 0.9141, test_Accuracy: 0.9201
Epoch: [ 0] [  449/  468] time: 48.4136, train_loss: 0.24977136, train_accuracy: 0.9609, test_Accuracy: 0.9205
Epoch: [ 0] [  450/  468] time: 48.5847, train_loss: 0.16166115, train_accuracy: 0.9609, test_Accuracy: 0.9217
Epoch: [ 0] [  451/  468] time: 48.6713, train_loss: 0.33457917, train_accuracy: 0.8828, test_Accuracy: 0.9221
Epoch: [ 0] [  452/  468] time: 48.7552, train_loss: 0.31712487, train_accuracy: 0.8984, test_Accuracy: 0.9227
Epoch: [ 0] [  453/  468] time: 48.8393, train_loss: 0.29505807, train_accuracy: 0.9062, test_Accuracy: 0.9231
Epoch: [ 0] [  454/  468] time: 48.9736, train_loss: 0.27919471, train_accuracy: 0.8906, test_Accuracy: 0.9232
Epoch: [ 0] [  455/  468] time: 49.0561, train_loss: 0.30718076, train_accuracy: 0.9062, test_Accuracy: 0.9244
E